# Load Packages

In [22]:
# For torch functions
import torch

# for Neural network layers 
import torch.nn as nn

# For neural network functions:
import torch.nn.functional as F

# For Open ML datasets available in pytorch
from torchvision import datasets, transforms

# for Optimization function in pytorch
import torch.optim as optim

# Load Train and Test Data

In [5]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,)),
                              ])
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

100.1%Extracting /Users/pvajja/.pytorch/MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to /Users/pvajja/.pytorch/MNIST_data/MNIST/raw
113.5%Downloading http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz to /Users/pvajja/.pytorch/MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz
Extracting /Users/pvajja/.pytorch/MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to /Users/pvajja/.pytorch/MNIST_data/MNIST/raw
180.4%Extracting /Users/pvajja/.pytorch/MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to /Users/pvajja/.pytorch/MNIST_data/MNIST/raw
Extracting /Users/pvajja/.pytorch/MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /Users/pvajja/.pytorch/MNIST_data/MNIST/raw
Processing...
Done!


In [47]:
testset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

# Create your own Neural Network for forward Pass

In [43]:
class NeuralNetwork(nn.Module):

    def __init__(self):
        super().__init__()

        # Define Layers:
        self.l1 = nn.Linear(784, 256)
        self.l2 = nn.Linear(256, 128)
        self.l3 = nn.Linear(128, 64)
        self.l4 = nn.Linear(64, 10)

        # Define Activation functions:
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
        self.softmax = nn.LogSoftmax(dim = 1)


    def forward(self, x):
        # Pass the input tensor x through our layers:
        x = self.l1(x)
        x = self.relu(x)
        x = self.l2(x)
        x = self.relu(x)
        x = self.l3(x)
        x = self.sigmoid(x)
        x = self.l4(x)
        x = self.softmax(x)

        return x

    
NN = NeuralNetwork()  # Intialize you NN

# Backpropagation

In [45]:
# Loss Function
criterion = nn.NLLLoss()

# Loss Otimizer
optimizer = optim.Adam(NN.parameters(), lr = 0.001)

# No:of times to train data
epochs = 5
for e in range(epochs):
    for images, labels in trainloader:
        # Faltten the images 
        images = images.view(images.shape[0], -1)

        # set optimizer gradients to zero:
        optimizer.zero_grad()

        
        output = NN(images) # Intial output
        loss = criterion(output, labels) # Loss  Caluclation
        loss.backward() # Pass loss function gradients to pervious layers:
        optimizer.step() # Update Weights

    print(loss.item()) # print loss for each epoch

0.047625500708818436
0.0713285580277443
0.018748214468359947
0.023736275732517242
0.032160669565200806


# Save your Model

In [46]:
PATH = './NeuralNet.pth'
torch.save(NN.state_dict(), PATH) 

# NN = NeuralNetwork()
# net.load_state_dict(torch.load(PATH))

#### To Load your network back:

NN = NeuralNetwork()

NN.load_state_dict(torch.load(PATH))


# Predict on Test Data

In [62]:
# Accuracy on Test Data
correct = 0
total = 0
with torch.no_grad():
    for images, labels in testloader:
        images = images.view(images.shape[0], -1)
        output = NN(images)
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 97 %


In [66]:
classes = ('0','1','2','3','4','5','6','7','8','9')
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.view(images.shape[0], -1)
        outputs = NN(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

# Accuracy of each class:
for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of     0 : 100 %
Accuracy of     1 : 100 %
Accuracy of     2 : 92 %
Accuracy of     3 : 98 %
Accuracy of     4 : 98 %
Accuracy of     5 : 90 %
Accuracy of     6 : 96 %
Accuracy of     7 : 98 %
Accuracy of     8 : 98 %
Accuracy of     9 : 96 %
